Assumptions:
1. User have similar rating in past will have similar rating in future.
2. For computing pearson similarity we consider only the rating which user 1 and user 2 both has value. If either of it is not available like "Nan" value that rating is not considered.
3. The preference of user stays consistent over time.
4. The movie rating for a target user is almost Nan which means only 5-10% we have values. So the missing ratings is first computed.
5. For predicting movie recommendations for a user we need to first predict the missing movie ratings of the user in target with the similar users (top 100 similar users is considered) first and then the ratings are sorted and top K is picked up and displayed to the user as top K recommended movies for user in target.
6. When a top similiar users doesnt have the movie rating which is considered for prediction then the target users movie rating's mean is returned and stored as the rating for that movie.
7. For predicting single user top k movies, 100 similar users were considered and top 200 movies are suggested for each user.
8. Each user in a considered group has top 200 movies recommended for them seperately and commonly rated movies by all 3 users in the group is retrieved.
9. These commonly rated movies are then sent to aggregation function such as weighted average and least misery for obtaining combined group recommendation for this specific group using hybrid aggregation.
10. Dataset is divided into j chunks depending on number of iteration sequence(j) required.
11. For suggesting the first set of recommendation we use the data chunk 1 and compute movie rating for the users in group and generate aggregated lists and we set alpha = 0 so that the hybrid aggregation function returns top 10 movies for round 1 completely from the weighted average aggregation list.
12. For computing the user satisfaction for each user from their top 200 recommended movies the first top K(eg: 10) movies is considered and the ratings for this movies alone are summed and used as the denominator. And for the numerator the group recommended movies which are present in top k list of that target user is taken but to sum up the movie ratings of the filtered common movies we will use the rating value available from the target users list.
13. For computing the disagreement "Coefficient of variation" method is used this is conputed for each j iteration.
14. For recommending the 2nd round of top k movie list this disagreement is considered. Utilising the 2nd chunk of dataset for predicting unknown movie ratings of the users in group and using aggregations proposed the aggregated list is formed and top k movies are displayed to the group and agin satisfaction and disagreement score is computed. It is repeated for next j iterations and top K recommended movies list are given to the group at the end of each iteration sequence.


Running Instruction:
1. u.data, u.item, u.user <- these are the 3 files needed for running this code.
2. Preferrably run the file in colab and have these 3 data files in drive.
3. Cells are in order so it can be run from top to bottom one by one.
4. For selecting users to form a group rather than the test input which is already given, there is a separate K means clustering function from which the users can be picked manually from a single cluster.
5. The **Testing** happens on the **2nd last code cell** of this notebook which has the comment line called "#testing" there we can vary the sequenceIteration required (j) and topMovies (K) which are currently set as 3 and 10 respectively depending on the assignment requirement.The group size is fixed so it should be 3 users all the time. If we want to test this for different set of users we can do that by editing the users=[] array and give the user id's inside this array



In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from google.colab import drive
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

In [2]:

drive.mount('/content/drive')
file_path_u_data = '/content/drive/My Drive/RS/u.data'
file_path_u_user = '/content/drive/My Drive/RS/u.user'
file_path_u_item = '/content/drive/My Drive/RS/u.item'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 **part 1  -  Importing data**

In [3]:
#updating respective variables with the data imported
column_names=["user_id", "item_id",  "rating", "timestamp"]
df=pd.read_table(file_path_u_data,header=None,names=column_names)
df = df.drop("timestamp", axis=1)
user_item_matrix = df.pivot(index="user_id", columns="item_id", values="rating")

col_names = {
    "item": ['movie id' , 'movie title' , 'release date' , 'video release date' ,
              'IMDb URL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
              "Children's" , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western'],
    "user": ['user id' , 'age' , 'gender' , 'occupation' , 'zip code']
}
movies = pd.read_table(file_path_u_item, header = None, sep='|', names = col_names["item"],  encoding='latin-1')
users = pd.read_table(file_path_u_user, header = None, sep='|', names = col_names["user"],  encoding='utf-8')


In [4]:
#finding similar users and forming groups

#since Knn doesnt accept Nan it is being replced with 0
user_item_matrix_without_nan = np.nan_to_num(user_item_matrix, nan=0)

# Number of clusters (groups) to create
num_clusters = 10

# Perform k-means clustering on the user-item matrix
kmeans = KMeans(n_clusters=num_clusters, random_state=30)

cluster_assignments = kmeans.fit_predict(user_item_matrix_without_nan)

# Identify the representative user for each cluster
representative_users = pairwise_distances_argmin_min(kmeans.cluster_centers_, user_item_matrix_without_nan)

# Print the clusters and representative users
for i in range(num_clusters):
    cluster_users = np.where(cluster_assignments == i)[0]
    print(f"\nCluster {i + 1} Users: {cluster_users}")
    print(f"Representative User for Cluster {i + 1}: {representative_users[0][i]}")



/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



Cluster 1 Users: [  2   3  28  32  33  34  35  38  39  45  46  60  73  85  87  99 104 106
 110 111 112 115 125 126 128 132 133 139 142 145 146 148 154 165 169 170
 172 178 190 203 204 205 219 228 239 240 257 259 260 272 277 280 281 283
 301 303 308 316 318 334 340 350 352 354 355 361 363 376 383 387 399 403
 407 409 413 417 422 426 427 430 439 442 443 445 447 450 460 461 463 481
 484 488 501 506 508 509 510 514 518 519 528 530 543 546 549 556 563 569
 573 577 586 588 595 597 610 615 625 627 630 634 645 651 655 667 672 674
 675 680 682 686 687 694 695 700 701 712 719 723 724 728 731 739 742 749
 751 754 771 774 781 782 783 786 790 800 801 802 807 808 809 810 811 812
 815 817 818 819 823 826 827 831 840 841 844 852 855 859 862 865 870 872
 897 904 914 919 925 930]
Representative User for Cluster 1: 925

Cluster 2 Users: [  7  21  36 101 109 118 144 157 177 196 216 220 248 255 266 319 327 331
 345 346 373 424 441 446 452 465 486 492 520 585 599 618 620 626 637 639
 670 704 708 714 745 75

Pearson similarity Function

In [5]:
#PEARSON SIMILARITY FUNCTION MANUAL IMPLEMENTATION

def pearson_correlation_computed_function(target_user, user2, user_item_matrix):
    # Find common items rated by both users
    common_items = (user_item_matrix.loc[target_user].dropna().index).intersection(user_item_matrix.loc[user2].dropna().index)
    if len(common_items) == 0:
        return 0  # No common items, so correlation is 0

    # Extract the ratings for the common items
    #r(a,p)
    user1_ratings = [user_item_matrix.loc[target_user][i] for i in common_items]
    #r(b,p)
    user2_ratings = [user_item_matrix.loc[user2][i] for i in common_items]

    # Calculate the mean ratings for both users
    # mean(r_a)
    mean_user1 = np.mean(user1_ratings)
    #  mean(r_b)
    mean_user2 = np.mean(user2_ratings)

    # Calculate the numerator and denominators for Pearson correlation
    # (Σp[(r(a,p) - mean(r_a)) * (r(b,p) - mean(r_b)]) for every p belongs to set "P" - which has common ratings from target user as well as user in consideration.
    numerator = sum((user1_ratings[i] - mean_user1) * (user2_ratings[i] - mean_user2) for i in range(len(common_items)))
    #  [√(Σ(r(a,p) - mean(r_a)² * √(Σ(r(b,p) - mean(r_b)²)]
    denominator_user1 = np.sqrt(sum((user1_ratings[i] - mean_user1) ** 2 for i in range(len(common_items))))
    denominator_user2 = np.sqrt(sum((user2_ratings[i] - mean_user2) ** 2 for i in range(len(common_items))))

    # Calculate the Pearson correlation coefficient
    if denominator_user1 == 0 or denominator_user2 == 0:
        return 0  # Handle division by zero
    else:
       #pearson correlation = (Σp[(r(a,p) - mean(r_a)) * (r(b,p) - mean(r_b)])/ [√(Σ(r(a,p) - mean(r_a)² * √(Σ(r(b,p) - mean(r_b)²)]
        correlation = numerator / (denominator_user1 * denominator_user2)
        return correlation

#pearson_similarity = pearson_correlation(user1, user2)
#print(f"Pearson Correlation: {pearson_similarity}")


#calls manually defined pearson function iteratively to compute similarity for all the user 942 users exlusing the target user.
#parameter explanation - (target_user) -> user for whom the recommendation is required, (num_users) -> consider num_users as a int n it is like returning top n closely similar user to the user in target.
def find_similar_users_with_compute_pearson(target_user, num_users,user_item_matrix):
    similarities = {}
    #iterating over each user in the user-item matrix
    for user in user_item_matrix.index:
        #condition to skip when the current user in the iteration is equal to the target user. Since we dont want to compute pearson similarity for a target user user the target user himself.
        if user != target_user:
            #computing similarity for target user eg:Alice with other user eg:user1(increased like user2, user3 till users 943 one by one) using predefined pearson function
            #similarity = pearson_correlation_predefined(target_user, user)
            #computing similarity for target user eg:Alice with other user eg:user1(increased like user2, user3 till users 943 one by one) using formula based computed pearson function
            similarity = pearson_correlation_computed_function(target_user, user, user_item_matrix)
            #similarities are stored in this array called "similarities" with respect to each user_id as as its index. Eg: similarity[102] gives the similarity value between the target user and the user 102.
            similarities[user] = similarity
    #filtered_similarity = {k: v for k, v in similarities.items() if not isinstance(v, float) or not math.isnan(v)}
    return sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:num_users]

#PREDICTION FUNCTION GIVEN IN CLASS


def prediction_funciton_inclass(targetUser,movie,pearsonResult100,meanRA,user_item_matrix):
    #defining how many neighbors should be considered for predicting movie rating
    #num_users=100
    #pearsonResult=find_similar_users_with_compute_pearson(targetUser, num_users)

    #similar users and their corresponding similarity values for top 100 users are computed and stored.
    #finding out of these similar users which users has the rated value for movie which the prediction should be done to the target user. Eg: if movie number 101 for the target user needs to be prediction even from the top 100 similar user to the target user is considered entirely. We should implement a filter which filters only the user which has rating for this secific movie.
    similarUsers=[item[0] for item in pearsonResult100]
    similarUserWithRating=[]
    for i in similarUsers:
      if(not math.isnan(user_item_matrix.loc[i][movie])):
        #print(user_item_matrix.loc[i][movie])
        similarUserWithRating.append(i)
    filteredPearsonResult=[]
    for item in pearsonResult100:
      if item[0] in similarUserWithRating:
        filteredPearsonResult.append(item)

    filteredPearsonResultArr = np.array(filteredPearsonResult)
    #if there is ratings for that particular movie with top 100 users then compute the below else return mean of RA
    if filteredPearsonResultArr.size != 0:
        #compute rest of the formula
        numer=0
        denom=0
        #print(filteredPearsonResult)
        #print("predicting movie...",movie)
        for simUser,simAB in filteredPearsonResult:
          common_items = (user_item_matrix.loc[targetUser].dropna().index).intersection(user_item_matrix.loc[simUser].dropna().index)
          if len(common_items) == 0:
              return 0  # No common items, so correlation is 0

          # Extract the ratings for the common items
          #r(b,p)
          rb_ratings = [user_item_matrix.loc[simUser][i] for i in common_items]

          #r_a(mean target user rating)
          ra_ratings = [user_item_matrix.loc[simUser][i] for i in common_items]

          # Calculate the mean ratings for both user
          #  mean(r_b)
          meanRB = np.mean(rb_ratings)

          #curret users movie rating for the speific movie in prediction
          Rbp=user_item_matrix.loc[simUser][movie]

          #computes the numerator part of the given predictor function (sum(sim(a,b)*(rbp-mean(r_b)))
          cal=(simAB*(Rbp-meanRB))
          numer+=cal
          #computes the denominator part of the given predictor function (sum(sim(a,b)))
          denom+=simAB

        #final computation of the predictor function
        #pred(a,p)=mean(r_a)+(sum(sim(a,b)*(rbp-mean(r_b)))/(sum(sim(a,b)))
        return meanRA+(numer/denom)
    else:
      return meanRA


#finding similar movies
def predict_topmovies_forSingleUser(selectedUser,topMovies,similarUser,user_item_matrix):
    print("predicting for user",selectedUser)
    user_ratings = user_item_matrix.loc[selectedUser]
    new_movie_recommendations = defaultdict(float)
    similar_users100 = find_similar_users_with_compute_pearson(selectedUser,similarUser,user_item_matrix)
    #r_a(mean target user rating)
    availableRatingInA=[]
    #filtering out rating in A which is the target user where rating in not "NaN"
    for i in range(1,len(user_item_matrix.loc[selectedUser])):
      if(not math.isnan(user_item_matrix.loc[selectedUser][i])):
        availableRatingInA.append(user_item_matrix.loc[selectedUser][i])
    meanRA=np.mean(availableRatingInA)

    for movie in user_item_matrix.columns:
        if pd.isna(user_ratings[movie]) or user_ratings[movie] == 0:
            predicted_rating = prediction_funciton_inclass(selectedUser, movie,similar_users100,meanRA, user_item_matrix)
            #predicted rating for the movie considered is stored in movie recommendation variable.
            if predicted_rating>5:
              predicted_rating=5
            else:
              predicted_rating
            new_movie_recommendations[movie] = predicted_rating
        #else:
            #the rating of the movie which the target user already has is also being append to the movie recommendation variable.
            #movie_recommendations[movie] = user_ratings[movie]
    return sorted(new_movie_recommendations.items(), key=lambda x: x[1], reverse=True)[:topMovies]

# Takes each member from the given group(list of similar user) and predicts top K movies fro each user.
def predict_topMovies_for_users(users,user_item_matrix):
    topMoviesToConsider=300
    topSimilarUserToConsider=100
    predictedRatings=[0]*len(users)
    i=0
    for user in users:
      predictedRatings[i]=(predict_topmovies_forSingleUser(user,topMoviesToConsider,topSimilarUserToConsider,user_item_matrix))
      i+=1

    return predictedRatings



In [6]:
# Average Aggregation method

def average_aggregation(users_top_common_movie_rating):
    # Find the average of the user ratings for a movie(average rating for each movie)

    avg_ratings = {}

    for singleUserRecomList in users_top_common_movie_rating:
        for movie_id, rating in singleUserRecomList:
            if movie_id not in avg_ratings:
                avg_ratings[movie_id] = rating
            else:
                avg_ratings[movie_id] = avg_ratings[movie_id]+rating

    # Convert the dictionary items back to a list of tuples
    avgResult = [(movie_id, min_rating/3) for movie_id, min_rating in avg_ratings.items()]


    return avgResult

In [17]:
# Weighted Average Aggregation method

def weighted_average_aggregation(users_top_common_movie_rating,user_item_matrix,users):
  Weighted_avg_ratings = {}

  #computing weights for each user based on the number of movies they had rated. Eg: if they has rated 65 movies their weight is 0.65 if other user had rated 20 movie his weight is 0.20
  weightOfUsers=[]
  for user in users:
    weightOfUsers.append(np.count_nonzero(~np.isnan(user_item_matrix.iloc[user,:]))/100)
  i=0
  for singleUserRecomList in users_top_common_movie_rating:
    #print(singleUserRecomList)

    weight = weightOfUsers[i]
    #print(weight)
    for movie_id, rating in singleUserRecomList:
      if movie_id not in Weighted_avg_ratings:
        Weighted_avg_ratings[movie_id] = weight*rating
      else:
        Weighted_avg_ratings[movie_id] = Weighted_avg_ratings[movie_id]+weight*rating
    i+=1
  totalWeight=sum(weightOfUsers)
  # Convert the dictionary items back to a list of tuples
  weightedAvgResult = [(movie_id, min_rating/totalWeight) for movie_id, min_rating in Weighted_avg_ratings.items()]

  return weightedAvgResult

In [8]:
# Least Misery Aggregation
def least_misery_aggregation(users_top_common_movie_rating):
    # Find the least satisfied user (min rating for each movie)
    # Create a dictionary to store the minimum rating for each movie id
    min_ratings = {}

    # Iterate through each sublist and update the dictionary with the minimum rating for each movie id
    for singleUserRecomList in users_top_common_movie_rating:
        for movie_id, rating in singleUserRecomList:
            if movie_id not in min_ratings or rating < min_ratings[movie_id]:
                min_ratings[movie_id] = rating

    # Convert the dictionary items back to a list of tuples
    least_satisfied_user = [(movie_id, min_rating) for movie_id, min_rating in min_ratings.items()]

    return least_satisfied_user

In [9]:
#Calculating Satisfaction scores for single user:

# when user satisfaction is 1, that person is completely satisfied. All the movies from their top K list are in the group recommendations top K list.
def computeUserSatisfactions(users_top_common_movie_rating,AggregatedOutputMovieIds,topMovies):
      #here we will sum up all the movie ratings available in the top K list for the target user
    singleUserSatisfactionScore=[]
    for singleUserRecom in users_top_common_movie_rating:
      singleUserRecom=sorted(singleUserRecom, key=lambda x: x[1],reverse=True)[:topMovies]
      #this is computed by summing up the ratings of the movies in the top K list which are common for target users top K list and also we will be summing up these moving rating value from the target user list not the rating value available in group prediction list.
      groupSatisfactionScore=0
      currentSingleUserSatisfactionScore=sum(x[1] for x in singleUserRecom)
      #print(singleUserRecom)
      #print(AggregatedOutputMovieIds)
      for movieId, rating in singleUserRecom:
        #print(movieId,rating)
        if(movieId in AggregatedOutputMovieIds):
          groupSatisfactionScore+=rating
        #print("groupSatisfactionScore")
        #print(groupSatisfactionScore)
        #print("currentSingleUserSatisfactionScore")
        #print(currentSingleUserSatisfactionScore)
      singleUserSatisfactionScore.append(groupSatisfactionScore/currentSingleUserSatisfactionScore)
    return singleUserSatisfactionScore


In [10]:
def hybrid_aggregation_model(AVG_aggregatedOutput,LM_aggregatedOutput,disagreement):
  avgAggregatedMovieListSorted=sorted(AVG_aggregatedOutput, key=lambda x: x[0], reverse=True)
  lmAggregatedMovieListSorted=sorted(LM_aggregatedOutput, key=lambda x: x[0], reverse=True)
  computedMovieList=[]
    #Formula to prepare an hybrid model
  for avgMovieItem,LMMovieItem in zip(avgAggregatedMovieListSorted,lmAggregatedMovieListSorted):
    avgMovieId,avgMovieRating = avgMovieItem
    lmMovieId,lmMovieRating = LMMovieItem
    #check whether the considered movie IDs are same from both list to avoid miscomputaion
    if(avgMovieId==lmMovieId):
      computedMovieList.append((avgMovieId,((1-disagreement)*avgMovieRating)+((disagreement)*lmMovieRating)))
  return sorted(computedMovieList, key=lambda x: x[1], reverse=True)


In [11]:
def compute_Disagreement(userSatisfactionScores):
  #Using Coefficient of Variation (CV) approach
  cv = (np.std(userSatisfactionScores) / np.mean(userSatisfactionScores))
  return cv


In [12]:
def predictTopCommonMoviesAndComputeAggregatedList(users,user_item_matrix):
  predictedRatings=predict_topMovies_for_users(users,user_item_matrix)
  first_elements_1 = {t[0] for t in predictedRatings[0]}
  first_elements_2 = {t[0] for t in predictedRatings[1]}
  first_elements_3 = {t[0] for t in predictedRatings[2]}
  common_movies= first_elements_1.intersection(first_elements_2, first_elements_3)
  filtered_list1 = sorted([t for t in predictedRatings[0] if t[0] in common_movies], key=lambda x: x[0])
  filtered_list2 = sorted([t for t in predictedRatings[1] if t[0] in common_movies], key=lambda x: x[0])
  filtered_list3 = sorted([t for t in predictedRatings[2] if t[0] in common_movies], key=lambda x: x[0])
  #print("User ",users[0],"common movie rating in top 200 movies")
  #print(filtered_list1)
  #print("User ",users[1],"common movie rating in top 200 movies")
  #print(filtered_list2)
  #print("User ",users[2],"common movie rating in top 200 movies")
  #print(filtered_list3)
  users_top_common_movie_rating=[filtered_list1,filtered_list2,filtered_list3]

  #Aggregating with Average aggregation
  #AVG_aggregatedOutput_unsorted=average_aggregation(users_top_common_movie_rating)

  #Aggregating with Weighted Average aggregation
  WEIGHTED_AVG_aggregatedOutput_unsorted=weighted_average_aggregation(users_top_common_movie_rating,user_item_matrix,users)

  #Aggregating using least misery
  LM_aggregatedOutput_unsorted=least_misery_aggregation(users_top_common_movie_rating)

  #return AVG_aggregatedOutput_unsorted,LM_aggregatedOutput_unsorted,users_top_common_movie_rating
  return WEIGHTED_AVG_aggregatedOutput_unsorted,LM_aggregatedOutput_unsorted,users_top_common_movie_rating

In [13]:
def chunkingDataset(num_chunks):

  totalUsers=len(user_item_matrix)
  user_chunk_size = totalUsers // num_chunks
  user_chunks = [user_chunk_size] * num_chunks
  countedUserChunk=sum(user_chunks)
  usersPerChunk = list(np.cumsum(user_chunks))

  #checking for missing user due to uneven splitting
  if(countedUserChunk!=totalUsers):
    usersPerChunk[num_chunks-1]=usersPerChunk[num_chunks-1]+(totalUsers-countedUserChunk)

  totalMovies=user_item_matrix.shape[1]
  movie_chunk_size = totalMovies // num_chunks
  movie_chunks = [movie_chunk_size] * num_chunks
  countedMovieChunk=sum(movie_chunks)
  moviesPerChunk = list(np.cumsum(movie_chunks))

  #checking for missing movie due to uneven splitting
  if(countedMovieChunk!=totalMovies):
    moviesPerChunk[num_chunks-1]=moviesPerChunk[num_chunks-1]+(totalMovies-countedMovieChunk)

  print("Chunking the Input Dataset\n")
  for i in range(len(usersPerChunk)):
    print("Dataset Chunk",i+1,"which has",usersPerChunk[i],"users and",moviesPerChunk[i],"movies.")

  print("\n")
  chunks = []

  # Loop through each chunk and create the submatrix
  for i in range(num_chunks):
      chunk = user_item_matrix.iloc[0:usersPerChunk[i], 0:moviesPerChunk[i]]
      chunks.append(chunk)
  return chunks


In [14]:
#sequence of group recommendation

def sequenceGroupRecommendation(iteration,users,topMovies):
  allGroupRecommendation=[]
  allRecommendedMovieID=[]
  groupRecommendationForCurrentIteration=[]
  topKMovieIdsForCurrentIteration=[]
  groupSatisfactions=[]
  allUserSatisfactionsForCurrentIteration=[]
  allGroupDisagreements=[]
  allMovieIds=[]

  #chunking Data
  chunks = chunkingDataset(iteration)

  for iter in range(iteration+1):
    if(iter==0):
      num_users, num_movies = chunks[iter].shape
      print("Predicting Recommendations for Sequence 1 for the given group of users",users,"using the splitted chunk 1 of the dataset which has",num_users,"users and",num_movies,"movies rated by them.\n")
      Weighted_aggregatedOutput_unsorted,LM_aggregatedOutput_unsorted,users_top_common_movie_rating=predictTopCommonMoviesAndComputeAggregatedList(users,chunks[iter])
      WeightedAggregatedMovieListSorted=sorted(Weighted_aggregatedOutput_unsorted, key=lambda x: x[1], reverse=True)
      lmAggregatedMovieListSorted=sorted(LM_aggregatedOutput_unsorted, key=lambda x: x[1], reverse=True)
      for avgMovie in WeightedAggregatedMovieListSorted[:topMovies]:
        groupRecommendationForCurrentIteration.append(avgMovie)
        allMovieIds.append(avgMovie[0])
        topKMovieIdsForCurrentIteration.append(avgMovie[0])
      #print(allMovieIds)
    else:
      userSatisfaction=computeUserSatisfactions(users_top_common_movie_rating,topKMovieIdsForCurrentIteration,topMovies)
      print("\n\nUser satisfaction scores from iteration sequence",iter,"is",userSatisfaction,"which is computed using round",iter,"recommended list")
      groupSatisfactions.append(np.mean(userSatisfaction))
      groupDisagreementForCurrentIteration=compute_Disagreement(userSatisfaction)
      print("User disagreement score from iteration sequence",iter,"is",groupDisagreementForCurrentIteration,"which is computed using round",iter,"recommended list")
      allGroupDisagreements.append(groupDisagreementForCurrentIteration)
      if (iter != iteration):
        num_users, num_movies = chunks[iter].shape
        print("\nPredicting Recommendations for Sequence",iter+1," for the given group of users",users,"using the splitted chunk",iter+1,"of the dataset which has",num_users,"users and",num_movies,"movies rated by them.\n")
        Weighted_aggregatedOutput_unsorted,LM_aggregatedOutput_unsorted,users_top_common_movie_rating=predictTopCommonMoviesAndComputeAggregatedList(users,chunks[iter])
        newmovieList=hybrid_aggregation_model(Weighted_aggregatedOutput_unsorted,LM_aggregatedOutput_unsorted,groupDisagreementForCurrentIteration)
        notYetRecommendedNewMovieList=[(t,p) for t,p in newmovieList if t not in allMovieIds]
        groupRecommendationForCurrentIteration=notYetRecommendedNewMovieList[0:topMovies]
        topKMovieIdsForCurrentIteration=[t for t,p in groupRecommendationForCurrentIteration]
        allMovieIds.append([t for t,p in groupRecommendationForCurrentIteration])
        allMovieIds = [item for sublist in allMovieIds for item in (sublist if isinstance(sublist, list) else [sublist])]
        #print(allMovieIds)
      else:
        continue
    print("\n\nTop",topMovies,"movies Recommended for the group which has users",users,"for the sequence itertion",iter + 1,"are")
    for movie, rating in groupRecommendationForCurrentIteration:
      print(f"Movie Name: {movies.loc[movie][1]}, Movie ID: {movie}, Predicted Rating: {rating:.0f}, Release Date: {movies.loc[movie][2]}")
  print("\nOverall Group Satisfaction after",iteration,"itertion of recommending top",topMovies,"movies is",np.mean(groupSatisfactions))
  print("\nThe Overall progression of group satisfaction for",iteration,"iterations are",groupSatisfactions)
  print("\nThe Overall progression of group disagreement for",iteration,"iterations are",allGroupDisagreements)


In [16]:
#testing
#different users can be selected but the current implementation only allows group of 3 users(fixed).
users=[145, 146, 148]
#users=[61,  63,  69]
#required sequence of recommendations can be mentioned here eg:3 iterations
sequenceIteration=3
#required top k results of movies can be mentioned here Eg:10 (will result top 10 recommended movies for the group at each sequence of recommendation).
topMovies=10
#compute sequential recommendation
sequenceGroupRecommendation(sequenceIteration,users,topMovies)


Chunking the Input Dataset

Dataset Chunk 1 which has 314 users and 560 movies.
Dataset Chunk 2 which has 628 users and 1120 movies.
Dataset Chunk 3 which has 943 users and 1682 movies.


Predicting Recommendations for Sequence 1 for the given group of users [145, 146, 148] using the splitted chunk 1 of the dataset which has 314 users and 560 movies rated by them.

predicting for user 145
predicting for user 146
predicting for user 148


Top 10 movies Recommended for the group which has users [145, 146, 148] for the sequence itertion 1 are
Movie Name: Batman & Robin (1997), Movie ID: 253, Predicted Rating: 5, Release Date: 20-Jun-1997
Movie Name: Delicatessen (1991), Movie ID: 170, Predicted Rating: 5, Release Date: 01-Jan-1991
Movie Name: Third Man, The (1949), Movie ID: 512, Predicted Rating: 5, Release Date: 01-Jan-1949
Movie Name: Lost World: Jurassic Park, The (1997), Movie ID: 251, Predicted Rating: 5, Release Date: 23-May-1997
Movie Name: Exotica (1994), Movie ID: 45, Predicted 